In [8]:
import random

class Individual:
    
    def __init__(self, nbUsers, nbResources, cost, fixedCost):
        self.code = []
        for i in range(nbResources):
            self.code.append(random.random() < 0.25)
        self.correctNonFeasible()
        self.fitness = self.fitnessFunction(cost, fixedCost)
    
    def __lt__(self, other):
        return self.fitness < other.fitness
    
    def correctNonFeasible(self):
        nbResources = len(self.code)
        for i in range(nbResources):
            if self.code[i]:
                return
        i = random.randrange(nbResources)
        self.code[i] = True
        
    def fitnessFunction(self, cost, fixedCost):
        nbUsers = len(cost)
        nbResources = len(fixedCost)
        value = 0.0
        used = [False]*nbResources
        for i in range(nbUsers):
            min = float('inf')
            k = -1
            for j in range(nbResources):
                if self.code[j] and cost[i][j] < min:
                    min = cost[i][j]
                    k = j
            value += min
            used[k] = True
        for i in range(nbResources):
            if used[i]:
                value += fixedCost[i]
        self.code = used
        return value

In [2]:
def readInput(filename):
    input = open(filename, "r")
    nbUsers, nbResources = [int(i) for i in input.readline().split()]
    cost = [[int(j) for j in input.readline().split()] for i in range(nbUsers)] 
    fixedCost = [int(i) for i in input.readline().split()]
    return nbUsers, nbResources, cost, fixedCost 

In [3]:
def selection(population):
    min = float('inf')
    k = -1
    for i in range(6):
        j = random.randrange(100)
        if population[j].fitness < min:
            min = population[j].fitness
            k = j
    return k

In [4]:
def crossover(parent1, parent2, child1, child2):
    nbResources = len(parent1.code)
    i = random.randrange(nbResources)
    for j in range(i):
        child1.code[j] = parent1.code[j]
        child2.code[j] = parent2.code[j]
    for j in range(i, nbResources):
        child1.code[j] = parent2.code[j]
        child2.code[j] = parent1.code[j]
    child1.correctNonFeasible()
    child2.correctNonFeasible()

In [5]:
def mutation(individual):
    nbResources = len(individual.code)
    for i in range(nbResources):
        if random.random() > 0.05:
            continue
        individual.code[i] = not individual.code[i]
    individual.correctNonFeasible()

In [11]:
nbUsers, nbResources, cost, fixedCost = readInput('uflp.txt')
population = []
newPopulation = []
for i in range(100):
    population.append(Individual(nbUsers, nbResources, cost, fixedCost))
    newPopulation.append(Individual(nbUsers, nbResources, cost, fixedCost)) 
for iteration in range(500):
    population.sort()
    for i in range(30):
        newPopulation[i] = population[i]
    for i in range(30, 100, 2):
        k1 = selection(population)
        k2 = selection(population)
        crossover(population[k1], population[k2], newPopulation[i], newPopulation[i + 1])
        mutation(newPopulation[i])
        mutation(newPopulation[i+1])
        newPopulation[i].fitness = newPopulation[i].fitnessFunction(cost, fixedCost)
        newPopulation[i + 1].fitness = newPopulation[i + 1].fitnessFunction(cost, fixedCost)
    population = newPopulation
population.sort()

print('Solution:', population[0].fitness)

Solution: 34.0


In [12]:


class Individual:
    
    def __init__(self, nbUsers, nbResources, cost, fixedCost):
        self.code = []
        for i in range(nbResources):
            self.code.append(random.random() < 0.25)
        self.correctNonFeasible()
        self.fitness = self.fitnessFunction(cost, fixedCost)
        
    def __lt__(self, other):
        return self.fitness < other.fitness
        
    def isFeasible(self):
        nbResources = len(self.code)
        for i in range(nbResources):
            if self.code[i]:
                return True
        return False
        
    def correctNonFeasible(self):
        nbResources = len(self.code)
        for i in range(nbResources):
            if self.code[i]:
                return
        i = random.randrange(nbResources)
        self.code[i] = True
        
    def invert(self):
        nbResources = len(self.code)
        i = random.randrange(nbResources)
        self.code[i] = not self.code[i]
        if self.isFeasible():
            return i
        self.code[i] = not self.code[i]
        return -1
        
    def fitnessFunction(self, cost, fixedCost):
        nbUsers = len(cost)
        nbResources = len(fixedCost)
        value = 0.0
        used = [False] * nbResources
        for i in range(nbUsers):
            min = float('inf')
            k = -1
            for j in range(nbResources):
                if self.code[j] and cost[i][j] < min:
                    min = cost[i][j]
                    k = j
            value += min
            used[k] = True
        for i in range(nbResources):
            if used[i]:
                value += fixedCost[i]
        self.code = used
        return value

    
def simulatedAnnealing(individual, maxIters, cost, fixedCost):
    for i in range(maxIters):
        j = individual.invert()
        if j < 0:
            continue
        newFitness = individual.fitnessFunction(cost, fixedCost)
        if newFitness < individual.fitness:
            individual.fitness = newFitness
        else:
            p = 1.0 / (i + 1) ** 0.5
            q = random.uniform(0, 1)
            if p > q:
                individual.fitness = newFitness
            else:
                individual.code[j] = not individual.code[j]


nbUsers, nbResources, cost, fixedCost = readInput('uflp.txt')
population = []
newPopulation = []
for i in range(100):
    population.append(Individual(nbUsers, nbResources, cost, fixedCost))
    newPopulation.append(Individual(nbUsers, nbResources, cost, fixedCost))
    
for iteration in range(500):
    population.sort()
    for i in range(30):
        newPopulation[i] = population[i]
    for i in range(30, 100, 2):
        k1 = selection(population)
        k2 = selection(population)
        crossover(population[k1], population[k2], newPopulation[i], newPopulation[i + 1])
        mutation(newPopulation[i])
        mutation(newPopulation[i+1])
        newPopulation[i].fitness = newPopulation[i].fitnessFunction(cost, fixedCost)
        newPopulation[i + 1].fitness = newPopulation[i + 1].fitnessFunction(cost, fixedCost)
    simulatedAnnealing(newPopulation[0], 10, cost, fixedCost)
    population = newPopulation
    
population.sort()
print('Solution:', population[0].fitness)



Solution: 34.0
